In [185]:
import asyncio
import pybotters
import pandas as pd
from loguru import logger

import pandas as pd
import datetime
import math
from typing import Tuple

In [186]:
# e.g. https://api.cryptowat.ch/markets/:exchange/:pair/ohlc
def generate_path(
        exchange: str,
        pair: str,
        min: int,
        unixTime: int) -> str:
    
    return f'https://api.cryptowat.ch/markets/{exchange}/{pair}/ohlc?periods={min}&after={unixTime}'

print(generate_path('bitflyer', 'btcfxjpy', 900, 1675368565))

https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods=900&after=1675368565


In [187]:
def str_to_timestamp(s: str) -> int:

    # str → datetime e.g.'2022-05-27 10:11:23'
    dt = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S")

    # datetime → timestamp
    ts = datetime.datetime.timestamp(dt)

    return math.floor(ts)

ts = str_to_timestamp('2022-05-27 10:11:23')
print(ts)

1653613883


In [188]:
def get_url(candle: int, date: str, size: int) -> Tuple[str, str]:
        url = 'https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods='
        candle = candle*60
        period = ('%s'%(candle))
        beforeurl = '&before='
        afterurl = '&after='
        # The maximum size of data is 6000 depending on a timing (sometimes you get less than 6000)
        before = str_to_timestamp(date)
        after = before - (candle * size)
        URL=('%s%s%s%s%s%s'%(url, candle, afterurl, after, beforeurl, before))
        # e.g https://api.cryptowat.ch/markets/bitflyer/btcjpy/ohlc?periods=86400&after=1483196400

        return URL, period

In [189]:
async def main(candle: int, date: str, size: int) -> pd.DataFrame:
    async with pybotters.Client() as client:
        URL, period = get_url(candle, date, size)

        response = await client.get(URL)
        body = await response.json()
        res = body['result']

        date, op, hi, lo, cl, volume = [], [], [], [], [], []

        for i in res[period]:
            date.append(i[0])
            op.append(i[1])
            hi.append(i[2])
            lo.append(i[3])
            cl.append(i[4])
            volume.append(i[5])

        date_datetime = map(datetime.datetime.fromtimestamp, date)
        dti = pd.DatetimeIndex(date_datetime)
        df = pd.DataFrame({"op" : op, "hi" : hi, "lo": lo, "cl" : cl, "volume" : volume}, index=dti)
        
        return df

In [190]:
if __name__ == '__main__':
    try:
        df = await main(15, '2023-02-02 20:09:25', 6000)
        display(df)
        print(df.dtypes)
        df.to_pickle('df_ohlcv.pkl')
    except KeyboardInterrupt:
        pass

[1670058900, 1670059800, 1670060700, 1670061600, 1670062500, 1670063400, 1670064300, 1670065200, 1670066100, 1670067000, 1670067900, 1670068800, 1670069700, 1670070600, 1670071500, 1670072400, 1670073300, 1670074200, 1670075100, 1670076000, 1670076900, 1670077800, 1670078700, 1670079600, 1670080500, 1670081400, 1670082300, 1670083200, 1670084100, 1670085000, 1670085900, 1670086800, 1670087700, 1670088600, 1670089500, 1670090400, 1670091300, 1670092200, 1670093100, 1670094000, 1670094900, 1670095800, 1670096700, 1670097600, 1670098500, 1670099400, 1670100300, 1670101200, 1670102100, 1670103000, 1670103900, 1670104800, 1670105700, 1670106600, 1670107500, 1670108400, 1670109300, 1670110200, 1670111100, 1670112000, 1670112900, 1670113800, 1670114700, 1670115600, 1670116500, 1670117400, 1670118300, 1670119200, 1670120100, 1670121000, 1670121900, 1670122800, 1670123700, 1670124600, 1670125500, 1670126400, 1670127300, 1670128200, 1670129100, 1670130000, 1670130900, 1670131800, 1670132700, 167

,op,hi,lo,cl,volume
2022-12-03 18:15:00,2295696,2298321,2294116,2297404,30.543160
2022-12-03 18:30:00,2297487,2298157,2296549,2298122,8.825385
2022-12-03 18:45:00,2298102,2298913,2297168,2297429,13.106698
2022-12-03 19:00:00,2297695,2300840,2297424,2300109,28.126075
2022-12-03 19:15:00,2300101,2300460,2299314,2299428,12.011023
...,...,...,...,...,...
2023-02-02 19:00:00,3076436,3080258,3074061,3079758,38.497302
2023-02-02 19:15:00,3079612,3080461,3076000,3077042,26.473347
2023-02-02 19:30:00,3076671,3078445,3075097,3076117,22.083219
2023-02-02 19:45:00,3076233,3076907,3071484,3074388,27.569929


op          int64
hi          int64
lo          int64
cl          int64
volume    float64
dtype: object
